In [2]:
import pdfplumber
import pandas as pd
import re
from IPython.display import display
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
import numpy as np
import faiss
import pickle
import os
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams

In [4]:
# Ensure stopwords are downloaded
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hemanthkumarbm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hemanthkumarbm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:

# Load the PDF file dynamically
pdf_path = "Cisco_FinanicalData_2023and2024.pdf" 

# Extract text from PDF
extracted_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        extracted_text += page.extract_text() + "\n"

# Print extracted raw text (First 1000 characters for preview)
print("Extracted Raw Text Preview:\n")
print(extracted_text[:1000])

# Split text into lines
lines = extracted_text.split("\n")

# Define structured data dictionary
structured_pdf_data = {
    "Category": [],
    "October 29, 2022": [], "January 28, 2023": [], "April 29, 2023": [], "July 29, 2023": [],
    "October 28, 2023": [], "January 27, 2024": [], "April 27, 2024": [], "July 27, 2024": []
}

# Convert dict_keys to list for indexing
date_columns = list(structured_pdf_data.keys())[1:]

# Process extracted text with improved parsing
for line in lines:
    # Remove unwanted characters like currency symbols, commas, and parentheses
    cleaned_line = re.sub(r"[\$,()]", "", line.strip())

    # Use '|' if available, otherwise split by multiple spaces
    parts = cleaned_line.split("|") if "|" in cleaned_line else re.split(r'\s{2,}', cleaned_line)
    parts = [p.strip() for p in parts]  # Clean spaces

    if len(parts) == 9 and "ASSETS" not in parts[0] and "LIABILITIES" not in parts[0]:
        category = parts[0]  # First column is the financial category
        values = parts[1:]   # Remaining columns are financial values

        structured_pdf_data["Category"].append(category)
        for i, date in enumerate(date_columns):
            structured_pdf_data[date].append(pd.to_numeric(values[i], errors="coerce") if i < len(values) else None)

# Convert to DataFrame
pdf_balance_sheet_df = pd.DataFrame(structured_pdf_data)

# Convert numeric columns to proper data types
for col in date_columns:
    pdf_balance_sheet_df[col] = pd.to_numeric(pdf_balance_sheet_df[col], errors="coerce")

# Add metadata
pdf_balance_sheet_df.attrs["Note"] = "All financial values are in million USD ($M)."

# Save cleaned data
pdf_balance_sheet_df.to_csv("Cleaned_Cisco_FinanicalData_2023and2024.csv", index=False)
pdf_balance_sheet_df.to_json("Cleaned_Cisco_FinanicalData_2023and2024.json", orient="records")

# Display the cleaned data properly in Jupyter Notebook
print("\n Final Cleaned Cisco Balance Sheet (Values in Million USD):\n")
display(pdf_balance_sheet_df.style.set_properties(**{'text-align': 'center'}).set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('background-color', '#f4f4f4')]}]
))


Extracted Raw Text Preview:

HISTORICAL FINANCIALS
BALANCE SHEETS
(in millions)
October 29, January 28, April 29, July 29, October 28, January 27, April 27, July 27,
2022 2023 2023 2023 2023 2024 2024 2024
ASSETS
Cash and cash equivalents ($ 7,292) ($ 9,009) ($ 8,044) ($ 10,123) ($ 9,602) ($ 13,715) ($ 8,913) ($ 7,508)
Investments ( 12,492) ( 13,052) ( 15,244) ( 16,023) ( 13,921) ( 11,956) ( 9,857) ( 10,346)
Accounts receivable, net ( 5,439) ( 5,237) ( 5,104) ( 5,854) ( 4,833) ( 4,884) ( 5,127) ( 6,685)
Inventories ( 2,664) ( 3,140) ( 3,474) ( 3,644) ( 3,342) ( 3,209) ( 3,118) ( 3,373)
Financing receivables, net ( 3,683) ( 3,557) ( 3,402) ( 3,352) ( 3,414) ( 3,476) ( 3,443) ( 3,338)
Other current assets ( 4,571) ( 4,520) ( 4,682) ( 4,352) ( 4,547) ( 4,887) ( 5,428) ( 5,612)
Total current assets ( 36,141) ( 38,515) ( 39,950) ( 43,348) ( 39,659) ( 42,127) ( 35,886) ( 36,862)
Property and equipment, net ( 1,972) ( 1,964) ( 2,047) ( 2,085) ( 2,004) ( 2,005) ( 2,000) ( 2,090)
Financing rece

,Category,"October 29, 2022","January 28, 2023","April 29, 2023","July 29, 2023","October 28, 2023","January 27, 2024","April 27, 2024","July 27, 2024"
0,Cash and cash equivalents,7292,9009,8044,10123,9602,13715,8913,7508
1,Investments,12492,13052,15244,16023,13921,11956,9857,10346
2,Accounts receivable net,5439,5237,5104,5854,4833,4884,5127,6685
3,Inventories,2664,3140,3474,3644,3342,3209,3118,3373
4,Financing receivables net,3683,3557,3402,3352,3414,3476,3443,3338
5,Other current assets,4571,4520,4682,4352,4547,4887,5428,5612
6,Total current assets,36141,38515,39950,43348,39659,42127,35886,36862
7,Property and equipment net,1972,1964,2047,2085,2004,2005,2000,2090
8,Financing receivables net,3618,3554,3393,3483,3324,3364,3251,3376
9,Goodwill,38160,38388,38369,38535,38900,39087,58633,58660


In [20]:
# Extract text from PDF
extracted_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        extracted_text += page.extract_text() + "\n"

# Tokenize words and filter financial terms
tokens = word_tokenize(extracted_text.lower())
filtered_tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words("english")]

# Count word frequency
word_counts = Counter(filtered_tokens)

# Get top 50 most frequent words (adjust based on dataset)
FINANCE_KEYWORDS = [word for word, count in word_counts.most_common(50)]

# Print extracted finance-related keywords
print("Extracted Finance Keywords from Balance Sheet:")
print(FINANCE_KEYWORDS)

Extracted Finance Keywords from Balance Sheet:
['assets', 'total', 'liabilities', 'net', 'current', 'deferred', 'equity', 'payable', 'october', 'january', 'april', 'july', 'cash', 'accounts', 'financing', 'receivables', 'debt', 'income', 'taxes', 'revenue', 'historical', 'financials', 'balance', 'sheets', 'millions', 'equivalents', 'investments', 'receivable', 'inventories', 'property', 'equipment', 'goodwill', 'purchased', 'intangible', 'tax', 'accrued', 'compensation', 'cisco', 'systems', 'inc']


In [22]:
def dataframe_to_text_chunks(df, section_name):
    """Converts DataFrame rows into separate year-based text chunks."""
    chunks = []
    
    for _, row in df.iterrows():
        category = row.iloc[0]  # The category (e.g., "Cash and cash equivalents")
        
        # Loop through each year's value and store separately
        for col in df.columns[1:]:  # Skip the first column (Category)
            year_value = row[col]
            if pd.notna(year_value):  # Ensure the value is not NaN
                chunks.append(f"{category}: {col}: {year_value}")

    return chunks

# Convert Balance Sheet into separate year-based text chunks
balance_sheet_chunks = dataframe_to_text_chunks(pdf_balance_sheet_df, "Balance Sheet")

# Convert into a DataFrame for better display
df_text_chunks = pd.DataFrame({"Extracted Text Chunks": balance_sheet_chunks})

# Display full column text without truncation
pd.set_option("display.max_colwidth", None)
display(df_text_chunks.head(20))  # Show first 10 rows


,Extracted Text Chunks
0,"Cash and cash equivalents: October 29, 2022: 7292"
1,"Cash and cash equivalents: January 28, 2023: 9009"
2,"Cash and cash equivalents: April 29, 2023: 8044"
3,"Cash and cash equivalents: July 29, 2023: 10123"
4,"Cash and cash equivalents: October 28, 2023: 9602"
5,"Cash and cash equivalents: January 27, 2024: 13715"
6,"Cash and cash equivalents: April 27, 2024: 8913"
7,"Cash and cash equivalents: July 27, 2024: 7508"
8,"Investments: October 29, 2022: 12492"
9,"Investments: January 28, 2023: 13052"


In [26]:
# Load a pre-trained embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for the balance sheet data
balance_sheet_embeddings = embedding_model.encode(
    balance_sheet_chunks, clean_up_tokenization_spaces=True
)

# Convert to NumPy array
balance_sheet_embeddings = np.array(balance_sheet_embeddings)

# Save embeddings and text chunks for future retrieval
np.save("Cisco_FinanicalData_embeddings.npy", balance_sheet_embeddings)
with open("Cisco_FinanicalData_chunks.txt", "w") as f:
    for chunk in balance_sheet_chunks:
        f.write(chunk + "\n")

print("Embeddings generated and saved successfully!")

/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embeddings generated and saved successfully!


In [28]:
# Create FAISS index
d = balance_sheet_embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(d)
index.add(balance_sheet_embeddings)

# Save FAISS index
faiss.write_index(index, "Cisco_FinanicalData_index.faiss")

print("FAISS vector database created successfully!")


FAISS vector database created successfully!


In [30]:
# Function to validate finance-related queries
def is_valid_finance_query(query):
    query_lower = query.lower()
    return any(keyword in query_lower for keyword in FINANCE_KEYWORDS)

In [32]:
def retrieve_financial_data(query, top_k=3):
    if not is_valid_finance_query(query):
        return {"error": "Invalid query. Please ask finance-related questions."}
        
    query_embedding = embedding_model.encode([query])
    D, I = index.search(query_embedding, top_k)  # Search FAISS index

    # Load text chunks
    with open("Cisco_FinanicalData_chunks.txt", "r") as f:
        text_chunks = f.readlines()

    # Extract the latest year dynamically from the query
    match = re.search(r"\b(20\d{2})\b", query)  # Extract year (e.g., 2024)
    query_year = match.group(0) if match else None

    filtered_results = []
    for i in I[0]:
        text = text_chunks[i].strip()

        if query_year:
            # Extract only the requested year's value from text
            year_match = re.search(rf"{query_year}:\s*\d+", text)
            if year_match:
                filtered_results.append(f"{text.split('|')[0]} | {year_match.group()}")  # Keep category + year value
        else:
            # If no year is found, return full text (fallback)
            filtered_results.append(text)

    return filtered_results

# Example query
query = "What is Cisco's total assets for January 2024?"
retrieved_data = retrieve_financial_data(query)

# Convert retrieved data into a DataFrame for better display
df_retrieved = pd.DataFrame({"Retrieved Financial Data": retrieved_data})

# Set column width to avoid truncation
pd.set_option("display.max_colwidth", None)

# Display the DataFrame neatly
display(df_retrieved)

,Retrieved Financial Data
0,"Total Assets: January 27, 2024: 101174 | 2024: 101174"
1,"Total current assets: January 27, 2024: 42127 | 2024: 42127"


In [34]:
# Tokenize text chunks for BM25
tokenized_chunks = [chunk.lower().split() for chunk in balance_sheet_chunks]

# Initialize BM25 model
bm25 = BM25Okapi(tokenized_chunks)

# Save BM25 index
with open("bm25_index.pkl", "wb") as f:
    pickle.dump(bm25, f)

In [36]:
# Load BM25 index
with open("bm25_index.pkl", "rb") as f:
    bm25 = pickle.load(f)

# Load FAISS index
faiss_index = faiss.read_index("Cisco_FinanicalData_index.faiss")

# Load embeddings for retrieval
balance_sheet_embeddings = np.load("Cisco_FinanicalData_embeddings.npy")

In [38]:
def extract_financial_phrases(text, min_count=2):
    """Extract frequently occurring bigrams (2-word financial phrases) from text."""

    text = re.sub(r"[^\w\s%-]", "", text.lower()) 

    tokens = word_tokenize(text)

    bigrams = list(ngrams(tokens, 2))
    bigram_counts = Counter(bigrams)

    frequent_bigrams = [
        "_".join(bigram) for bigram, count in bigram_counts.items()
        if count >= min_count and (bigram[0] in FINANCE_KEYWORDS or bigram[1] in FINANCE_KEYWORDS)
    ]

    return frequent_bigrams

finance_phrases = extract_financial_phrases(extracted_text, min_count=2)

print("Extracted Financial Multi-Word Terms:", finance_phrases)

Extracted Financial Multi-Word Terms: ['financing_receivables', 'receivables_net', 'other_current', 'current_assets', 'total_current', 'liabilities_and', 'and_equity', 'income_taxes', 'taxes_payable', 'deferred_revenue', 'current_liabilities', 'total_liabilities']


In [40]:
def preprocess_query(query):
    """Replace finance-related multi-word phrases dynamically."""
    query = query.lower()

    for phrase in finance_phrases:
        phrase_with_spaces = phrase.replace("_", " ")  # Convert "total_assets" → "total assets"
        query = query.replace(phrase_with_spaces, phrase)  # Replace "total assets" → "total_assets"

    return query

In [42]:
def hybrid_retrieve(query, top_k=5, bm25_weight=0.4, faiss_weight=0.6):
    """Retrieve relevant results using BM25 and FAISS, only if query is valid."""
    
    # **Input Validation Guardrail**
    if not is_valid_finance_query(query):
        return ["Invalid query. Please ask finance-related questions."]

    query_cleaned = preprocess_query(query)
    query_embedding = embedding_model.encode([query])

    D, I = faiss_index.search(query_embedding, top_k)
    faiss_results = [(balance_sheet_chunks[i], 1 - D[0][idx]) for idx, i in enumerate(I[0])]

    query_tokens = word_tokenize(query_cleaned)
    bm25_scores = bm25.get_scores(query_tokens)
    bm25_top_idxs = np.argsort(bm25_scores)[-top_k:]
    bm25_results = [(balance_sheet_chunks[i], bm25_scores[i]) for i in bm25_top_idxs]

    merged_results = {}
    for text, score in faiss_results:
        merged_results[text] = merged_results.get(text, 0) + score * faiss_weight
    for text, score in bm25_results:
        merged_results[text] = merged_results.get(text, 0) + score * bm25_weight

    ranked_results = sorted(merged_results.items(), key=lambda x: x[1], reverse=True)

    year_pattern = re.search(r"\b(20\d{2})\b", query)  
    if year_pattern:
        target_year = year_pattern.group(0)
        ranked_results = [res for res in ranked_results if target_year in res[0]]

    return [res[0] for res in ranked_results[:top_k]]

# Example query (asking specifically for January 2024)
query = "What is Goodwill for January 2024?"
retrieved_data = hybrid_retrieve(query)

# Convert retrieved data into DataFrame for better display
df_retrieved = pd.DataFrame({"Hybrid Retrieved Financial Data": retrieved_data})

# Display neatly
pd.set_option("display.max_colwidth", None)
display(df_retrieved)

,Hybrid Retrieved Financial Data
0,"Goodwill: January 27, 2024: 39087"
1,"Investments: January 27, 2024: 11956"
2,"Inventories: January 27, 2024: 3209"
3,"Goodwill: July 27, 2024: 58660"


In [64]:
# Load Cross-Encoder model
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def is_hallucinated_response(response):
    """Check if AI response lacks key financial terms or contains random hallucinations."""
    contains_number = any(char.isdigit() for char in response)
    contains_financial_keyword = any(keyword in response.lower() for keyword in FINANCE_KEYWORDS)

    return not (contains_number and contains_financial_keyword)

def rerank_results(query, retrieved_texts):
    """Re-rank retrieved results using Cross-Encoder model and normalize scores properly."""
    
    if "Invalid query" in retrieved_texts[0]:  # If the query was invalid, return the error message
        return retrieved_texts

    query_pairs = [[query, text] for text in retrieved_texts]  
    scores = cross_encoder.predict(query_pairs)

    # Normalize scores using Min-Max Scaling to avoid 100% saturation
    min_score, max_score = min(scores), max(scores)
    if max_score > min_score:  # Avoid division by zero
        scores = [(s - min_score) / (max_score - min_score) * 100 for s in scores]
    else:
        scores = [50 for _ in scores]  # Default to 50% if all scores are the same

    ranked_results = sorted(zip(retrieved_texts, scores), key=lambda x: x[1], reverse=True)

    # Output validation guardrail
    validated_results = []
    for res, score in ranked_results:
        if is_hallucinated_response(res):
            validated_results.append("AI-generated response is unreliable. Please refine your query.")
        else:
            validated_results.append(f"{res} (Score: {score:.2f}%)")

    return validated_results

def rag_pipeline(query):
    """Full RAG pipeline with input filtering & output validation."""

    # **Step 1: Validate input query**
    retrieved_data = hybrid_retrieve(query)

    # **Step 2: Generate AI response (Only if valid)**
    final_response = rerank_results(query, retrieved_data)

    return final_response
    
# Example usage
query = "What is Inventories in July 2024?"
reranked_data = rag_pipeline(query)

# Convert into DataFrame for better display
df_reranked = pd.DataFrame({"Re-Ranked Financial Data": reranked_data})

# Display results
display(df_reranked)


/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,Re-Ranked Financial Data
0,"Inventories: July 27, 2024: 3373 (Score: 100.00%)"
1,"Inventories: January 27, 2024: 3209 (Score: 51.41%)"
2,"Investments: July 27, 2024: 10346 (Score: 0.00%)"
